# Construction of a public dataset of Blood Bowl matches played on FUMBBL.com

This blogpost is about **Blood Bowl**, a boardgame I started playing last year. The goal of this blog post is to use Python API and HTML scraping to fetch online Blood Bowl match outcome data, and to create a structured dataset ready for analysis and visualization. 

In 2020, a new version of the Blood Bowl board game came out with several changes to the rules, and to the available teams to play with.
Last september, the online Blood Bowl gaming website **FUMBBL** switched to the new ruleset ("BB2020"), with players largely abandoning the previous ruleset (from 2016, "BB2016" hereafter). With a daily game volume of a few hundred matches, I decided it would be interesting to analyse the win rates of the different teams (or "races", given the fantasy world setting), and how these have been impacted by the new ruleset. This is important, because races can get **nerfed** (made weaker) or **buffed** (made stronger) when new versions of the ruleset are released. These changes are common in any gaming community, in an attempt to achieve more balance in the game. We examine whether the current "three tier" system to balance the strength differences between teams is optimal. The results can inform changes that improve balance.


# What is a data paper?

*Data publication is becoming increasingly important to the scientific community, as it will provide a mechanism for those who create data to receive academic credit
for their work and  will allow the conclusions arising from an analysis to be more readily verifiable, thus promoting transparency in the scientific process. Peer review of data will also provide a mechanism for ensuring the quality of datasets, and we provide suggestions on the types of activities one expects to see in the peer review of data. (Lawrence et al 2011)*

To advance science, there is a need to share data. For example, to be able to reproduce the results of others, and to build upon work of others.
Unfortunately, there are several issues with sharing data (Kratz & Strasser 2015):

* Ethical issues (Human subjects data, privacy laws etc)
* Time consuming: It takes time to carefully document the data: how it was collected, how it was processed, how the quality was assessed.
* Career risk: Others might use the shared dataset to write publications that could have been yours to write.

Contents of a data paper: motivation / dataset construction / dataset validation / re-use potential


Kratz & Strasser, Researcher Perspectives on Publication and Peer Review of Data
Lawrence et al, Citation and Peer Review of Data: Moving Towards Formal Data Publication

https://arxiv.org/abs/2004.03688 With the open dataset published on Zenodo and the documentation on Arxiv.org

# Legal stuff FUMBBL


All users of the FUMBBL.com website have agreed to the [terms of use](ref).

```
You explicitly agree, in using this web site or any service provided, that you shall not:
[...]
(c) collect or harvest any data about other users;
```

Part of the `terms of use` is the [privacy policy](https://fumbbl.com/p/privacy):

```
Content you provide through the website
All the information you provide through the website is processed by FUMBBL. This includes things such as forum posts, private message posts, blog entries, team and player names and biographies and news comments. Data provided this way is visible by other people on the website and in most cases public even to individuals without accounts (not including private messages), and as such are considered of public interest. If direct personal information is posted in public view, you can contact moderators to resolve this. Match records are also considered content in this context, and is also considered of public interest. This data is collected as the primary purpose of the website and it is of course entirely up to you how much of this is provided to FUMBBL. 

Third party sharing
Some of the public data is available through a public (*i.e. unauthenticated*) API, which shares some of the information provided by FUMBBL users in a way suitable for third-party websites and services to process.

The data available through the unauthenticated API is considered non-personal as it only reflects information that is public by its nature on the website. The authenticated API will only show information connected to the authenticated account.

You have already accepted these privacy rules. Should you change your mind, please go to the "Right to Erasure" page linked above. 
```

# Python packages used in this blog post

This blogpost is written as a Jupyter notebook, and is fully reproducible. The idea is to make Blood Bowl data analysis accessible to others. Using only open source tooling reduces the barriers for others to take what i created and build on it. Apart from having Python installed, you need a code editor. Jupyter Notebooks can be read by many code editors, such as Jupyterlab, or Pycharm, i decided to try out [Visual Studio Code](https://code.visualstudio.com/), or **VS code** for short. It ticks all my boxes such as being cross platform (I use linux at home and Windows at work), being open source, and having a large user base, as [visible on Github](https://github.com/microsoft/vscode) with more than 125K stars. It works pretty well, the only thing I really miss is have a [Python console connected to the same kernel my Notebook is running on](https://stackoverflow.com/questions/54987778/is-it-possible-to-link-the-interactive-python-window-to-a-running-jupyter-notebo).

In [77]:
%connect_info

{
  "shell_port": 49859,
  "iopub_port": 42075,
  "stdin_port": 51499,
  "control_port": 36057,
  "hb_port": 35409,
  "ip": "127.0.0.1",
  "key": "ce6c8061-cfbd3f3094ba574bbf050b47",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-42a5e190-771f-43f9-a08a-03b7d89163c0.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [78]:
import random
import time

import requests # API library

import pandas as pd
pd.set_option('display.max_rows', 500)

pd.set_option('display.max_columns', 500)

import numpy as np
import plotnine as p9

#import statsmodels.api as sm # statistics library

# Function for computing confidence intervals
from statsmodels.stats.proportion import proportion_confint   


# Blood Bowl online: FUMBBL 

The **FUMBBL** website (https://fumbbl.com) is one big pile of data. From coach pages, with their teams, to team rosters, with players, and match histories. It's all there.

To obtain **FUMBBL** data, we need to fetch it match by match, team by team. To do so, the site creator Christer Kaivo-oja, from Sweden, has made an API that allows us to easily fetch data. What follows is a short demonstration how the API works, before we fetch the **FUMBBL** match and team data of the last 12 months.
 

# Behold, the power of Requests / Using Python to fetch FUMBBL data

We use the [Python **Requests** library](https://docs.python-requests.org/en/latest/) to make the API call over HTTPS and obtain the response from the FUMBLL server. The response is in the JSON format, a [light-weight data-interchange format](https://www.json.org/json-en.html) which is both easy to read and write for humans, and easy to parse and generate by computers. So this makes it a natural choice for an API.

Here is an example of what is available at the coach level (in Blood Bowl, people playing the game are called *coaches*, since the playing pieces are already called the *players*). The full documentation of the API can be found at (https://fumbbl.com/apidoc/).


In [118]:
response = requests.get("https://fumbbl.com/api/coach/teams/gsverhoeven")
# display the complete JSON object {}
response.json()

{'id': 255851,
 'name': 'gsverhoeven',
 'teams': [{'id': 1003452,
   'coachId': 255851,
   'name': 'Hillywood Hellraisers',
   'rosterId': 50,
   'race': 'Human',
   'teamValue': 1090000,
   'canLfg': 'Yes',
   'isLfg': 'Yes',
   'games': '3',
   'divisionId': 1,
   'division': 'Ranked',
   'leagueId': 0,
   'league': None,
   'status': 'Retired',
   'raceLogos': [{'size': 32, 'logo': 486290},
    {'size': 48, 'logo': 486291},
    {'size': 64, 'logo': 486292},
    {'size': 96, 'logo': 486293},
    {'size': 128, 'logo': 486294},
    {'size': 192, 'logo': 486295}]},
  {'id': 1035833,
   'coachId': 255851,
   'name': 'Pharaoh Munchers',
   'rosterId': 51,
   'race': 'Tomb Kings',
   'teamValue': 960000,
   'canLfg': 'Yes',
   'isLfg': 'No',
   'games': '0',
   'divisionId': 1,
   'division': 'Ranked',
   'leagueId': 0,
   'league': None,
   'status': 'Active',
   'raceLogos': [{'size': 32, 'logo': 486296},
    {'size': 48, 'logo': 486297},
    {'size': 64, 'logo': 486298},
    {'size': 96

# Parsing JSON data

Let's have a closer look at the JSON data structure here.
We have a list of key-value pairs. 
Some keys contain simple values, such as `name`, 

In [84]:
response.json()['name']

'gsverhoeven'

but some return as value a new list of key-value pairs, such as `teams`.
Actually this is a "list of "lists of key-value pairs", since we have a separate list for each team.
Even the list of a single team contains new structure, for example under the key `raceLogos`.

In [85]:
response.json()['teams'][2]['raceLogos']

[{'size': 32, 'logo': 486290},
 {'size': 48, 'logo': 486291},
 {'size': 64, 'logo': 486292},
 {'size': 96, 'logo': 486293},
 {'size': 128, 'logo': 486294},
 {'size': 192, 'logo': 486295}]

In [86]:
response.json()['teams'][2]['name']

'Blackbox Bastards'

# What data do we need? And in what shape?

Now we know how the data comes in, we need to think about which variables we want, and how to structure them.
The most straightforward level to analyze race strength is to look at **match outcomes**.
At its core, the data consists of matches played by teams, commanded by coaches.
Furthermore, we expect race strength to change over time, as new strategies are discovered by the players, or new rules get introduced. So the time dimension is important as well.

So, let's go with a flat data frame with **rows for each match**, and columns for the various variables associated with each match.
These would include:

* Coach ids
* Team races
* Team ids
* Date of the match
* Outcome (Touchdowns of both teams)

With this basic structure, we can add as many match related variables in the future, keeping the basic structure (each row is a match) unchanged.

So lets get the match data!



# Step 1: Getting the match data

So we are mostly intested in the current ruleset, this is `BB2020`. This ruleset became available in **FUMBBL** at september 1st 2021, and two months later, some 5000 games have been played. We also want to compare with the previous ruleset, where we have much more data available. How far do we go back? 
Lets start with matches played during the last year. So starting from september 1st, 2020, up to oktober 1st, 2021. 
This way, we have roughly 12 months of `BB2016` ruleset matches, and one month of `BB2020` matches.

The easiest way to collect match data over a particular period of time is to just loop over `match_id`. The most recent match was 4.334.456, and since rougly 100.000 matches are played each year, we can fiddle about and we find match 4.226.550 played on september 1st, 2020.  So that means we need to collect some 110K matches. 

**VERY IMPORTANT: We do not want to overload the **FUMBBL** server, so we make only three API requests per second. In this way, the server load is hardly affected and it can continue functioning properly for all the Blood Bowl coaches playing their daily games!**

To collect 110K matches, we will need 110000*0.333/3600 = 15 hours.

In [87]:
# estimated hours fetching data
(4339204-4216257)*0.333/3600

11.372597500000001

In [88]:
df_matches = pd.DataFrame(columns=['match_id', 'match_date', 'match_time',  
    'team1_id', 'team1_coach_id', 'team1_roster_id', 'team1_race_name', 'team1_value',
    'team2_id', 'team2_coach_id', 'team2_roster_id', 'team2_race_name', 'team2_value',
    'team1_score', 'team2_score'])

target = 'data/df_matches_' + time.strftime("%Y%m%d_%H%M%S") + '.h5'
print(target)

end_match = 4347800
begin_match = 4339205
n_matches = end_match - begin_match
full_run = 0
print(n_matches)

if(full_run):
    for i in range(n_matches):
        api_string = "https://fumbbl.com/api/match/get/" + str(end_match - i)
        # wait 0.5 s on average between each API call
        wait_time = (random.uniform(0.5, 1) + 0.25)/3
        time.sleep(wait_time)
        match = requests.get(api_string)
        match = match.json()
        if match: # fix for matches that do not exist
            match_id = match['id']
            match_date = match['date']
            match_time = match['time']
            team1_id = match['team1']['id']
            team2_id = match['team2']['id']
            team1_score = match['team1']['score']
            team2_score = match['team2']['score']  
            team1_roster_id = match['team1']['roster']['id']
            team2_roster_id = match['team2']['roster']['id']            
            team1_coach_id = match['team1']['coach']['id']
            team2_coach_id = match['team2']['coach']['id']
            team1_race_name = match['team1']['roster']['name'] 
            team2_race_name = match['team2']['roster']['name'] 
            team1_value = match['team1']['teamValue']
            team2_value = match['team2']['teamValue']
            #print(match_id)     
            df_matches.loc[i] = [match_id, match_date, match_time, 
                team1_id, team1_coach_id, team1_roster_id, team1_race_name, team1_value,
                team2_id, team2_coach_id, team2_roster_id, team2_race_name, team2_value,
                team1_score, team2_score]
        else:
            # empty data for this match, create empty row
            match_id = int(end_match - i)
            df_matches.loc[i] = [np.NaN, np.NaN, np.NaN, 
            np.NaN,np.NaN,np.NaN,np.NaN,
            np.NaN,np.NaN,np.NaN,np.NaN,
            np.NaN,np.NaN, np.NaN, np.NaN] # try np.repeat([np.NaN], 13, axis=0) next time
            df_matches.loc[i]['match_id'] = int(match_id)
        if i % 100 == 0: 
            # write tmp data as hdf5 file
            print(i, end='')
            print(".", end='')
            df_matches.to_hdf(target, key='df_matches', mode='w')

    # write data as hdf5 file
    df_matches.to_hdf(target, key='df_matches', mode='w')
else:
    # read from hdf5 file
    df_matches1 = pd.read_hdf('data/df_matches_20211130_231815.h5')
    df_matches2 = pd.read_hdf('data/df_matches_20211106_205843.h5')
    df_matches = pd.concat([df_matches1, df_matches2], ignore_index=True)
#
df_matches.shape

data/df_matches_20211214_231407.h5
8595


(131542, 15)

In [89]:
# convert object dtype columns to proper pandas dtypes datetime and numeric
df_matches['match_date'] = pd.to_datetime(df_matches.match_date) # Datetime object
df_matches['match_id'] = pd.to_numeric(df_matches.match_id) 
df_matches['team1_id'] = pd.to_numeric(df_matches.team1_id) 
df_matches['team1_coach_id'] = pd.to_numeric(df_matches.team1_coach_id) 
df_matches['team1_roster_id'] = pd.to_numeric(df_matches.team1_roster_id) 
df_matches['team2_id'] = pd.to_numeric(df_matches.team2_id) 
df_matches['team2_coach_id'] = pd.to_numeric(df_matches.team2_coach_id) 
df_matches['team2_roster_id'] = pd.to_numeric(df_matches.team2_roster_id) 
df_matches['team1_score'] = pd.to_numeric(df_matches.team1_score) 
df_matches['team2_score'] = pd.to_numeric(df_matches.team2_score) 

# calculate match score difference
df_matches['team1_win'] = np.sign(df_matches['team1_score'] - df_matches['team2_score'])
df_matches['team2_win'] = np.sign(df_matches['team2_score'] - df_matches['team1_score'])

# mirror match
df_matches['mirror_match'] = 0
df_matches.loc[df_matches['team1_race_name'] == df_matches['team2_race_name'], 'mirror_match'] = 1



In [90]:
# 5K mirror matches
df_matches.query('mirror_match == 1')

# Fix Khorne team name
df_matches.loc[df_matches['team1_race_name'] == "Khorne", 'team1_race_name'] = 'Daemons of Khorne'
df_matches.loc[df_matches['team2_race_name'] == "Khorne", 'team2_race_name'] = 'Daemons of Khorne'

In [91]:
df_matches.query('team1_race_name == "Daemons of Khorne"')

,match_id,match_date,match_time,team1_id,team1_coach_id,team1_roster_id,team1_race_name,team1_value,team2_id,team2_coach_id,team2_roster_id,team2_race_name,team2_value,team1_score,team2_score,team1_win,team2_win,mirror_match
5,4347795,2021-11-30,21:33:55,1057305.0,226515.0,5916.0,Daemons of Khorne,1080k,1057277.0,259988.0,5142.0,Chaos Dwarf,1110k,1.0,0.0,1.0,-1.0,0
10,4347790,2021-11-30,21:20:03,1057021.0,42441.0,5916.0,Daemons of Khorne,830k,1044820.0,38724.0,4977.0,Snotling,860k,0.0,2.0,-1.0,1.0,0
22,4347778,2021-11-30,20:56:51,1057305.0,226515.0,5916.0,Daemons of Khorne,1040k,1054916.0,18576.0,4958.0,Chaos Renegade,1050k,1.0,0.0,1.0,-1.0,0
31,4347769,2021-11-30,20:28:10,1057021.0,42441.0,5916.0,Daemons of Khorne,1000k,1057682.0,245151.0,4962.0,Goblin,930k,0.0,2.0,-1.0,1.0,0
48,4347752,2021-11-30,19:23:00,1055088.0,64853.0,5916.0,Daemons of Khorne,1520k,1054971.0,39364.0,5916.0,Daemons of Khorne,1580k,1.0,0.0,1.0,-1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130507,4217292,2020-08-04,20:02:33,986649.0,243863.0,1720.0,Daemons of Khorne,1360k,988466.0,124683.0,56.0,Ogre,1060k,0.0,1.0,-1.0,1.0,0
130681,4217118,2020-08-04,04:14:04,991004.0,134237.0,3258.0,Daemons of Khorne,1090k,990291.0,252089.0,53.0,Necromantic Horror,1120k,2.0,0.0,1.0,-1.0,0
130703,4217096,2020-08-04,01:53:25,994378.0,248944.0,3304.0,Daemons of Khorne,1000k,994513.0,253040.0,3303.0,Chaos Renegade,1020k,2.0,1.0,1.0,-1.0,0
131055,4216744,2020-08-03,02:19:21,984860.0,235574.0,1720.0,Daemons of Khorne,1150k,987260.0,240845.0,72.0,Simyin,1350k,1.0,1.0,0.0,0.0,0


# Dataprep: transforming the team values

In Blood Bowl, teams can develop themselves over the course of multiple matches. The winnings of each match can be spend on buying new, stronger players, or replace the players that ended up getting injured or even killed. In addition, players receive so-called *star points* for important events, such as scoring, or inflicting a casualty on the opponent. Therefore, a balancing mechanism is needed when when a newly created "rookie" team is facing a highly developed opponent with lots of extra skills and strong players. 
Blood Bowl solves this by calculating for both teams their **Current team value**.
The **Team value difference** for a match determines the amount of gold that the weaker team can use to buy so-called **inducements**.
These inducements are temporary, and can consists of a famous "star player" who joins the team just for this match. Another popular option is to hire a wizard that can be used to turn one of the opposing players into a frog.

It is well known that the win rates of the teams depend on how developed a team is. For example, Amazons are thought to be strongest at low team value, as they already start out with lots of *block* and *dodge* skills, whereas a Chaos team start out with almost no skills.
So if we compare win rates, we would like take into account the current team value. 
Now as this can differ between the two teams in a match up, I reasoned that the highest team value is most informative about the average development level of both teams, because of the inducement mechanism described above.

In the dataset, we have for each match the current team values of both teams. Importantly, we do not have information on inducements.
We discuss later what this means for the conclusions we can draw. Here we transform the text string `1100k` into an integer number `1100`, so that we can calculated the difference, and pick for each match the maximum team value and store it as `tv_match`.


In [92]:
df_matches.dtypes

match_id                    int64
match_date         datetime64[ns]
match_time                 object
team1_id                  float64
team1_coach_id            float64
team1_roster_id           float64
team1_race_name            object
team1_value                object
team2_id                  float64
team2_coach_id            float64
team2_roster_id           float64
team2_race_name            object
team2_value                object
team1_score               float64
team2_score               float64
team1_win                 float64
team2_win                 float64
mirror_match                int64
dtype: object

In [93]:

# convert team value 1100k to 1100 integer and and above / below median (= low / high TV)
df_matches['team1_value'] = df_matches['team1_value'].str.replace('k$', '')
df_matches['team1_value'] = df_matches['team1_value'].fillna(0).astype(np.int64)

df_matches['team2_value'] = df_matches['team2_value'].str.replace('k$', '')
df_matches['team2_value'] = df_matches['team2_value'].fillna(0).astype(np.int64)

df_matches['tv_diff'] = np.abs(df_matches['team2_value'] - df_matches['team1_value'])

df_matches['tv_match'] = df_matches[["team1_value", "team2_value"]].max(axis=1)

In [94]:
# 131K matches
df_matches

,match_id,match_date,match_time,team1_id,team1_coach_id,team1_roster_id,team1_race_name,team1_value,team2_id,team2_coach_id,team2_roster_id,team2_race_name,team2_value,team1_score,team2_score,team1_win,team2_win,mirror_match,tv_diff,tv_match
0,4347800,2021-11-30,21:45:48,1057678.0,134383.0,4975.0,Shambling Undead,990,1054764.0,221827.0,4957.0,Chaos Chosen,1010,2.0,0.0,1.0,-1.0,0,20,1010
1,4347799,2021-11-30,21:42:50,1042263.0,255911.0,4966.0,Lizardmen,1260,1044174.0,248304.0,5146.0,Vampire,1130,1.0,0.0,1.0,-1.0,0,130,1260
2,4347798,2021-11-30,21:41:09,1047161.0,229369.0,5141.0,Amazon,1210,1041159.0,228582.0,4956.0,Black Orc,1060,1.0,1.0,0.0,0.0,0,150,1210
3,4347797,2021-11-30,21:38:12,1054766.0,79054.0,4964.0,Human,1270,1057416.0,251165.0,5916.0,Daemons of Khorne,1120,1.0,0.0,1.0,-1.0,0,150,1270
4,4347796,2021-11-30,21:34:08,1052653.0,18576.0,4961.0,Elven Union,1370,1057241.0,254700.0,4956.0,Black Orc,1300,3.0,0.0,1.0,-1.0,0,70,1370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131537,4216262,2020-08-01,22:53:07,995283.0,235272.0,364.0,Daemons of Nurgle,1000,995327.0,238300.0,395.0,Skink,980,0.0,2.0,-1.0,1.0,0,20,1000
131538,4216261,2020-08-01,22:51:25,990875.0,254533.0,54.0,Norse,1230,994045.0,134383.0,53.0,Necromantic Horror,1230,0.0,3.0,-1.0,1.0,0,0,1230
131539,4216260,2020-08-01,22:50:14,983323.0,245537.0,41.0,Amazon,1390,988905.0,250129.0,55.0,Nurgle,1680,0.0,1.0,-1.0,1.0,0,290,1680
131540,4216259,2020-08-01,22:50:05,989053.0,254630.0,47.0,Goblin,940,967266.0,250588.0,50.0,Human,920,0.0,2.0,-1.0,1.0,0,20,940


# Step 2: HTML Scraping the inducements and coach rankings for each match

I highly recommend [this tutorial](https://hackersandslackers.com/scraping-urls-with-beautifulsoup/) for a great introduction to `BeautifulSoup`.

In addition, to clean up the scraped text, I used **re** (Regular expressions), part of the [Python standard library](https://docs.python.org/3/library/index.html) to extract the actual inducements from the text string that contains them.

The next trick is to use `pandas` *Explode* method (similar to `separate_rows` in `tidyverse` R) to give each inducement its own row in the dataset.



In [95]:
from bs4 import BeautifulSoup
import re

df_inducements = pd.DataFrame(columns=['match_id', 'team1_inducements', 'team2_inducements', 'coach1_ranking', 'coach2_ranking'])

target = 'data/df_inducements_' + time.strftime("%Y%m%d_%H%M%S") + '.h5'
print(target)

end_match = 4216258  
begin_match = 4216257
n_matches = end_match - begin_match

full_run = 0

print(n_matches)

if(full_run):
    for i in range(n_matches):
        match_id = end_match - i
        api_string = "https://fumbbl.com/FUMBBL.php?page=match&id=" + str(match_id)
        # wait 0.5 s on average between each GET call
        wait_time = (random.uniform(0.5, 1) + 0.25)/2
        time.sleep(wait_time)
        response = requests.get(api_string)

        soup = BeautifulSoup(response.content, 'html.parser')

        if soup.find("div", {"class": "matchrecord"}) is not None:
            # match record is available
            inducements = soup.find_all("div", class_="inducements")

            pattern = re.compile(r'\s+Inducements: (.*)\n')

            match = re.match(pattern, inducements[0].get_text())
            if match:
                team1_inducements = match.group(1)
            else:
                team1_inducements = ''

            match = re.match(pattern, inducements[1].get_text())
            if match:
                team2_inducements = match.group(1)
            else:
                team2_inducements = ''

            coach_rankings = soup.find_all("div", class_="coach")

            coach1_ranking = coach_rankings[0].get_text()
            coach2_ranking = coach_rankings[1].get_text()

            df_inducements.loc[i] = [match_id, team1_inducements, team2_inducements, coach1_ranking, coach2_ranking]

        if i % 100 == 0: 
                    # write tmp data as hdf5 file
                    print(i, end='')
                    print(".", end='')
                    df_inducements.to_hdf(target, key='df_inducements', mode='w')

    # write data as hdf5 file
    df_inducements.to_hdf(target, key='df_inducements', mode='w')
else:
    # read from hdf5 file
    
    df_inducements0 = pd.read_hdf('data/df_inducements_20211210_094223.h5')
    df_inducements1 = pd.read_hdf('data/df_inducements_20211209_085450.h5')
    df_inducements2 = pd.read_hdf('data/df_inducements_20211209_234850.h5')
    df_inducements3 = pd.read_hdf('data/df_inducements_20211210_164630.h5')
    df_inducements4 = pd.read_hdf('data/df_inducements_20211211_124503.h5')
    df_inducements5 = pd.read_hdf('data/df_inducements_20211211_234148.h5')
   
    df_inducements = pd.concat([df_inducements0, df_inducements1, df_inducements2, df_inducements3, df_inducements4, df_inducements5], ignore_index = True)
    

df_inducements["match_id"].min() # moeten tot 4216258, missen 4216258


data/df_inducements_20211214_231411.h5
1


4216258

# Dataprep match record for each team

This creates again a dataframe that is double in size, `df_wins` because each match generates two rows.

PM WHAT DO WE NEED INDUCEMENTS FOR? IT IS NOT SAVED AT THE MOMENT

In [96]:
team1_inducements = df_inducements[['match_id', 'team1_inducements']]
team2_inducements = df_inducements[['match_id', 'team2_inducements']]

# make column names equal
team1_inducements.columns = team2_inducements.columns = ['match_id', 'inducements']

# row bind the two dataframes
inducements = pd.concat([team1_inducements, team2_inducements], ignore_index = True)

# convert comma separated string to list
inducements['inducements'] = inducements['inducements'].str.split(',')

# make each element of the list a separate row
inducements = inducements.explode('inducements')

# strip leading and trailing whitespaces
inducements['inducements'] = inducements['inducements'].str.strip()

# create "star player" label
inducements['star_player'] = 0
inducements.loc[inducements['inducements'].str.contains("Star player"), 'star_player'] = 1

# create "card" label
inducements['special_card'] = 0
inducements.loc[inducements['inducements'].str.contains("Card"), 'special_card'] = 1


# Dataprep: coach rankings

We want to extract the part `CR 152.53` from the scraped coach information field. Just as we matches on `Inducements:`, we can match on `CR ` and grab the contents directly after that, stopping when we encounter a whitespace.

We first play around a bit and test until we discover the proper RE to use :-)


In [97]:
#pattern = re.compile(r'\s+Inducements: (.*)\n')
pattern = re.compile(r'.*CR (.*)\s\(.*')

match = re.match(pattern, df_inducements.loc[0, 'coach2_ranking'])

if match is not None:
    print(match.group(1)) # group(0) is the whole string
else:
    print("match is none")
#df_inducements['coach1_CR'] = 
df_inducements.loc[0, 'coach2_ranking']

144.51


'LoxodonP Veteran CR 144.51 (-0.76)'

In [98]:
# Dataprep: add the coach rankings
df_inducements['coach1_CR'] = df_inducements['coach1_ranking'].str.extract(r'.*CR (.*)\s\(.*')
df_inducements['coach2_CR'] = df_inducements['coach2_ranking'].str.extract(r'.*CR (.*)\s\(.*')

df_inducements['coach1_CR'] = pd.to_numeric(df_inducements['coach1_CR'])
df_inducements['coach2_CR'] = pd.to_numeric(df_inducements['coach2_CR'])

df_inducements['CR_diff'] = np.abs(df_inducements['coach1_CR'] - df_inducements['coach2_CR'])

# Add inducement info to df_matches

PM HERE INDUCEMENTS IS USED TO DERIVE THE HAS_SP FIELD

In [99]:
df_matches = pd.merge(df_matches, df_inducements, on='match_id', how='left')

df_sp = (inducements
            .groupby("match_id")
            .agg(has_sp = ("star_player", "max"))
            .reset_index()
)


df_matches = pd.merge(df_matches, df_sp, on = "match_id", how = "left")

PM Add Coach ranking info to df_matches and df_wins



In [100]:
# make two copies, one for each team in the match
team1_wins = df_matches[['match_id', 'match_date', 'team1_id', 'team1_coach_id', 'team1_race_name', 'team1_value', 'team1_win', 'tv_diff', 'tv_match', 'mirror_match']].copy()
team2_wins = df_matches[['match_id', 'match_date',  'team2_id', 'team2_coach_id', 'team2_race_name', 'team2_value', 'team2_win', 'tv_diff', 'tv_match', 'mirror_match']].copy()

team1_wins.columns = team2_wins.columns = ['match_id', 'match_date', 'team_id', 'coach_id', 'race_name', 'team_value', 'wins', 'tv_diff', 'tv_match', 'mirror_match']

# combine both dataframes
df_wins = pd.concat([team1_wins, team2_wins])

# Match outcomes: How much is a draw worth?

If we want to calculate a win rate for each race, we need to decide what to do with draws.
I've never given the matter much thought, but it turns out other people have!

For example, in football, there is a popular weighting scheme where wins are given 3 points, draws 1 point and losses 0 points.
This is called [three points for a win](https://en.wikipedia.org/wiki/Three_points_for_a_win) . 
In Blood bowl leagues and tournaments, this rule is often used as well.
So, if we want to predict which teams perform best in these settings (have the highest probability of winning a tournament), we need to weigh the match outcomes accordingly.

However, in Blood Bowl data analysis, it seems that a 2:1:0 (W / D / L) weighting scheme is most commonly used. 
This scheme has the advantage that the weighted average win percentage over all matches is always 50%, creating a nice reference point allowing conclusions such as this and that team has an x percent above average win percentage.

For example, Mike Davies from the NAF calculates win rate by weighting each win as 1 point, and each draw as 0.5 points (For example, [here](https://public.tableau.com/app/profile/mike.sann0638.davies/viz/NAFGames_0/SuccessBB2020)).

So if we want to compare with others, it makes sense to adapt this scheme as well.

Now that we have a dataset on the match-team level, lets prep the data a bit further.
We add the half point for a draw in the `wins` column.

And we create a team value bin to be able to compare win rates for teams that are hopefully more similar in team development.


In [101]:
df_wins.loc[df_wins['wins'] == 0, 'wins'] = 0.5
df_wins.loc[df_wins['wins'] == -1, 'wins'] = 0

# convert to float
df_wins['wins'] = df_wins['wins'].astype(float)

df_wins['tv_bin'] = pd.cut(df_wins['tv_match'], [0, 950, 1250,1550, 1850, float("inf")], labels=['< 950', '1.1K', '1.4K', '1.7K', '> 1850'])


At this point, Lets have a look at our dataset again:

In [102]:
df_wins.query('team_value > 1000 & team_value < 1020')


,match_id,match_date,team_id,coach_id,race_name,team_value,wins,tv_diff,tv_match,mirror_match,tv_bin
8,4347792,2021-11-30,1057447.0,244443.0,Dwarf,1010,0.0,10,1020,0,1.1K
30,4347770,2021-11-30,1056368.0,259383.0,Nurgle,1010,1.0,40,1010,0,1.1K
93,4347707,2021-11-30,1057536.0,260358.0,Shambling Undead,1010,0.0,110,1120,0,1.1K
106,4347694,2021-11-30,1054308.0,221100.0,Northern Nippon,1010,0.0,20,1030,0,1.1K
137,4347663,2021-11-30,1047012.0,131680.0,Dwarf,1010,1.0,70,1080,0,1.1K
...,...,...,...,...,...,...,...,...,...,...,...
130555,4217244,2020-08-04,660739.0,4120.0,Vampire,1010,0.5,10,1020,0,1.1K
130767,4217032,2020-08-03,987634.0,220153.0,Underworld Denizens,1010,0.0,180,1190,0,1.1K
130912,4216887,2020-08-03,990543.0,44438.0,Goblin,1010,1.0,10,1010,0,1.1K
131071,4216728,2020-08-03,993666.0,253575.0,Clan Pestilens,1010,0.0,30,1040,0,1.1K


Great! Almost there. There is still something missing though, we need to know, for all the teams in our matches dataset, in what division or league they are playing, and what version of the rules they use. For these we turn to the API again, to fetch more data, now on the team level.

# Step 3: Fetch data on team division and ruleset

Let grab for all teams in `df_wins` the team **division** and **ruleset**.

A limitation of the API is that it shows only the latest version of the teams and leagues.  This hides the fact that leagues have changed their rules since they were first created. For example, the NAF used BB2016 rules up until summer of 2021, and thereafter switched to the new BB2020 ruleset for their latest online tournament.
So we have to use our "domain knowledge" here to interpret the data properly.

**Note to self**: contact Christer suggesting change to freeze rulesets as soons as matches have been played with them.
And force users to create new rulesets if they want to update / change previous rulesets.

In [103]:
# make list of all teams that need to be fetched
team_ids = list(df_wins.query('match_id < 4339205')['team_id'].dropna())

# get unique values by converting to a Python set and back to list
team_ids = list(set(team_ids))

len(team_ids)

43747

In [104]:
new_team_ids = list(df_wins.query('match_id >= 4339205')['team_id'].dropna())
team_ids = list(set(new_team_ids) - set(team_ids)) 
len(team_ids)

3084

So we have to fetch data for 43+4K different teams. 

We use the same approach as above, looping over all `team_id` 's and making a separate API call for each team.

**IMPORTANT: here too, we limit ourselves to a maximum of 3 API calls per second to avoid overloading the FUMBBL server**

In [105]:
df_teams = pd.DataFrame(columns=['team_id', 'division_id', 'division_name',  'league' ,
    'ruleset', 'roster_id', 'race_name',  'games_played'])

target = 'data/df_teams_' + time.strftime("%Y%m%d_%H%M%S") + '.h5'
print(target)

fullrun = 0

if fullrun:
    print('fetching team data for ', len(team_ids), ' teams')
    for t in range(len(team_ids)):    
        api_string = "https://fumbbl.com/api/team/get/" + str(int(team_ids[t]))
        wait_time = (random.uniform(0.5, 1) + 0.25)/3
        time.sleep(wait_time)
        team = requests.get(api_string)
        team = team.json()
        # grab fields
        team_id = team['id']
        division_id = team['divisionId']
        division_name = team['division']
        ruleset = team['ruleset']
        league = team['league']
        roster_id = team['roster']['id']
        race_name = team['roster']['name']
        games_played = team['record']['games']
        # add to dataframe
        df_teams.loc[t] = [team_id, division_id, division_name, league, ruleset, roster_id, race_name, games_played]
        if t % 100 == 0: 
            # write tmp data as hdf5 file
            print(t, end='')
            print(".", end='')
            df_teams.to_hdf(target, key='df_teams', mode='w')
    
    df_teams.to_hdf(target, key='df_teams', mode='w')
else:
    # read from hdf5 file
    df_teams1 = pd.read_hdf('data/df_teams_20211030_115137.h5')
    df_teams2 = pd.read_hdf('data/df_teams_20211202_222458.h5')
    df_teams = pd.concat([df_teams1, df_teams2], ignore_index=True)


df_teams['roster_name'] = df_teams['roster_id'].astype(str) + '_' + df_teams['race_name']

df_teams.shape

data/df_teams_20211214_231417.h5


(46831, 9)

# Dataprep: Add ruleset_version and division_name

FUMBBL allows coaches to create their own rulesets to play their own leagues and tournaments with. For example, there is a so-called "Secret League" where coaches can play with "Ninja halflings", of with "Ethereal" spirits etc. 

Since we want the team strength for the official rulesets BB2016 and BB2020, we need to distinguish those matches from the matches that are played under different rules.

Lets have look at the various divisions and leagues, which rulesets are used, and which races are played how often.
There are a lot of small leagues being played on FUMBBL, they account for maybe X% of all the matches.

We only look at divisions and leagues with a sufficient volume of matches, or otherwise we do not have sufficient statistics for each race.

So I aggregated the data by division, league and ruleset, and filtered on at least 150 different teams that have played at least once last year.
Apart from the main "Divisions" that are part of FUMBBL, there were a few user-run leagues present in this table, so I looked up their names on FUMBBL and what ruleset is used (BB2016, BB2020 or some other variant). This information (contained in an xlsx) is added to the dataset below.


In [106]:
# add ruleset_version and division_name from xlsx
ruleset_division_names = pd.read_excel('data/ruleset_division_names.xlsx',  engine='openpyxl')

df_teams = pd.merge(df_teams, ruleset_division_names, on= ['league', 'ruleset', 'division_id'], how='left')

df_teams['division_name'] = df_teams['new_division_name']

df_teams = df_teams.drop('new_division_name', 1)


In [107]:
(df_teams
    .groupby(['ruleset', 'league', 'division_id', 'division_name',  'ruleset_version'], dropna=False)
    .agg( n_teams = ('ruleset', 'count')
    )
    .sort_values('n_teams', ascending = False)
    .query('n_teams > 150')['n_teams']
    .reset_index()
)

,ruleset,league,division_id,division_name,ruleset_version,n_teams
0,1,0,10,Blackbox,bb2016,10439
1,1,0,1,Ranked,bb2016,7944
2,4,0,2,Competitive,bb2020,6532
3,6,0,5,Regular_league,bb2016,5942
4,303,10263,5,Secret League,bb2016,2554
5,2228,9298,5,NAF,mixed,1466
6,2,0,3,Stunty Leeg,bb2016,607
7,2198,14708,5,SL BB2020,bb2020,452
8,888,11676,5,LegaGladio,mixed,345
9,4,14713,5,Test Open League BB2020,bb2020,296


PM MOVE THIS PART TO THE ANALYSIS BLOG

We are primary interested in the new BB2020 ruleset version. We see that apart from the new **Competitive** division of FUMBBL itself, already a few user run leagues have started to use BB2020 rules. However, I chose to not include these leagues in the comparison: 

* Both NAF and Lega Gladio tournaments consist of a mix of BB2016 and BB2020 matches, making these more difficult to analyse. 
* Then there is the Secret League, which is different because of all the extra teams available there. 
* Test Open League BB2020 has likely been the test group before the Competitive Division became available.

The NAF tournaments current ruleset is interesting, because it introduces (for the first time) tiers in FUMBBL!
The NAF online tournament allows coaches to distribute a fixed number of skills, depending on the team's tier.
This makes match outcomes in this league less comparable to other divisions on FUMBBL.

Then on to the BB2016 ruleset. Here we have three big FUMBBL divisions: Blackbox, Ranked and (regular) League.
Blackbox and Ranked use ruleset 1. FUMBBL has a nice display of a rulesets, see e.g. here for [ruleset 1](https://fumbbl.com/p/ruleset?id=1). Comparing this ruleset to the ruleset used in the BB2016 regular league, we find a few small differences: e.g. the latter has the special play cards, has a few extra teams available (e.g. Simyin), but does not allow for wizards and does not the use the expensive mistakes rule.

To get sufficient observations for all the 25+ teams, we need at least a few thousands matches played.

So we end up with comparing

* Competitive BB2020

VERSUS

* Ranked BB2016
* Black box BB2016
* standard BB2016 League division (ruleset 6 / league 0)

If the differences between these three division can be considered to be small, we can pool the matches from these three leagues to get better statistics.
So lets compare win rates between these three divisions. To do so we add the division information to the `df_wins` data.

# Dataprep: Merging the match data with the team / ruleset data

For each match in the `df_wins` **DataFrame** we can now add the team-level information from `df_teams`.

As both datasets contain 'race_name', we drop one of them.

In [108]:
df_wins = pd.merge(df_wins, df_teams.drop('race_name', 1), on='team_id', how='left')

# 66 empty matches: we drop these
df_wins = df_wins.dropna(subset=['match_date'])

# Step 4: adding team tiers

We also add the team tiers we used for the NAF dataset.

In [109]:
race_tiers = pd.read_excel('data/race_tiers_mapping.xlsx',  engine='openpyxl')
race_tiers = race_tiers[ ['race_name', 'bb2020_tier', 'naf_tier', 'bb2020_nov21_tier']]
race_tiers = race_tiers.dropna()

In [110]:
# add bb2020 tiers
df_wins = pd.merge(df_wins, race_tiers, on='race_name', how='left')

df_wins['team_id'] = pd.to_numeric(df_wins.team_id) 

# Dataprep: getting the dates right

To see time trends, its useful to aggregate the data by week. For this we add a week number for each date, and from this week number, we convert back to a date to get a week_date. This last part is useful for plotting with `plotnine`, as this treats dates in a special way (PM how)
We use the ISO definition of week, this has some unexpected behavior near the beginning / end of each year. 

The data starts in week 36 (september) of 2020, and stops halfway week 44 in 2021. 

PM THINK IF WE WANT THIS IN DF_MATCHES AS WELL
PM FIX THE WEEK NUMBER DTYPE


In [111]:
df_wins['week_number'] = df_wins['match_date'].dt.isocalendar().week


# cannot convert to np int64 becaues of missing values
# cannot convert to 'int64'-dtype NumPy array with missing values. Specify an appropriate 'na_value' for this dtype.
df_wins['week_number'] = df_wins['week_number'].astype('Int64')
#df_wins['week_number'] = df_wins['week_number'].fillna(0).astype(np.int64)

# add year based on match date (but want it based on match ISO week)
df_wins['year'] = pd.DatetimeIndex(df_wins['match_date']).year

# manual fix year for ISO week 2020-53 (2020 has 53 ISO weeks, including a few days in jan 2021)
df_wins.loc[(df_wins['year'] == 2021) & (df_wins['week_number'] == 53), 'year'] = 2020

df_wins['week_year'] = df_wins['year'].astype(str) + '-' + df_wins['week_number'].astype(str)

df_wins['week_date'] = pd.to_datetime(df_wins['week_year'].astype("string") + '-1', format = "%Y-%U-%w")

# manual fix of week date (grrrr)
df_wins.loc[(df_wins['week_date'] ==  '2021-01-04') & (df_wins['week_number'] == 53), 'week_date'] = pd.to_datetime('2020-12-31')


In [112]:
df_matches.query("match_id == 4216260")


df_matches.head()




,match_id,match_date,match_time,team1_id,team1_coach_id,team1_roster_id,team1_race_name,team1_value,team2_id,team2_coach_id,team2_roster_id,team2_race_name,team2_value,team1_score,team2_score,team1_win,team2_win,mirror_match,tv_diff,tv_match,team1_inducements,team2_inducements,coach1_ranking,coach2_ranking,coach1_CR,coach2_CR,CR_diff,has_sp
0,4347800,2021-11-30,21:45:48,1057678.0,134383.0,4975.0,Shambling Undead,990,1054764.0,221827.0,4957.0,Chaos Chosen,1010,2.0,0.0,1.0,-1.0,0,20,1010,,,CR 152.53 (+0.76) Emerging Star Sp00keh,LoxodonP Veteran CR 144.51 (-0.76),152.53,144.51,8.02,0.0
1,4347799,2021-11-30,21:42:50,1042263.0,255911.0,4966.0,Lizardmen,1260,1044174.0,248304.0,5146.0,Vampire,1130,1.0,0.0,1.0,-1.0,0,130,1260,,"1 bloodweiser babe, Mercenary Merc Thrall Line...",CR 154.6 (+0.97) Emerging Star Vini,motay666 Emerging Star CR 151.12 (-0.97),154.60,151.12,3.48,0.0
2,4347798,2021-11-30,21:41:09,1047161.0,229369.0,5141.0,Amazon,1210,1041159.0,228582.0,4956.0,Black Orc,1060,1.0,1.0,0.0,0.0,0,150,1210,,"2 bribes, 1 bloodweiser babe",CR 164.82 (-0.01) Super Star Kurjo,Joost Super Star CR 164.74 (+0.01),164.82,164.74,0.08,0.0
3,4347797,2021-11-30,21:38:12,1054766.0,79054.0,4964.0,Human,1270,1057416.0,251165.0,5916.0,Daemons of Khorne,1120,1.0,0.0,1.0,-1.0,0,150,1270,,Star player Max Spleenripper,CR 168.24 (+0.48) Super Star Waterboy,KingCrow Emerging Star CR 151.24 (-0.48),168.24,151.24,17.00,1.0
4,4347796,2021-11-30,21:34:08,1052653.0,18576.0,4961.0,Elven Union,1370,1057241.0,254700.0,4956.0,Black Orc,1300,3.0,0.0,1.0,-1.0,0,70,1370,2 bribes,"2 bribes, 1 extra team training session",CR 161.63 (+0.35) Super Star dieter_h.,XpherAndTheAxes Veteran CR 141.38 (-0.35),161.63,141.38,20.25,0.0


# Save all four prepped datasets


In [113]:
# PM
target = 'data/df_teams_final.h5'
df_matches.to_hdf(target, key='df_matches', mode='w')

target = 'data/df_wins_final.h5'
#df_wins.to_hdf(target, key='df_wins', mode='w')

#df_teams
#df_inducements

/home/gertjan/venvs/requests_env/lib/python3.6/site-packages/pandas/core/generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block3_values] [items->Index(['match_id', 'match_time', 'team1_race_name', 'team2_race_name',
       'team1_inducements', 'team2_inducements', 'coach1_ranking',
       'coach2_ranking'],
      dtype='object')]



In [116]:
# cannot save week_number Integer array and tv_bin (category) to HDF
df_wins.drop(['week_number'], axis = 1, inplace = True)
df_wins.drop(['tv_bin'], axis = 1, inplace = True)

df_wins.to_hdf(target, key='df_wins', mode='w')

In [ ]:
df_wins.info()

In [117]:
df_wins

,match_id,match_date,team_id,coach_id,race_name,team_value,wins,tv_diff,tv_match,mirror_match,division_id,division_name,league,ruleset,roster_id,games_played,roster_name,ruleset_version,bb2020_tier,naf_tier,bb2020_nov21_tier,year,week_year,week_date
0,4347800,2021-11-30,1057678.0,134383.0,Shambling Undead,990,1.0,20,1010,0,2,Competitive,0,4,4975,3,4975_Shambling Undead,bb2020,1.0,1.0,1.0,2021,2021-48,2021-11-29
1,4347799,2021-11-30,1042263.0,255911.0,Lizardmen,1260,1.0,130,1260,0,2,Competitive,0,4,4966,6,4966_Lizardmen,bb2020,1.0,1.0,1.0,2021,2021-48,2021-11-29
2,4347798,2021-11-30,1047161.0,229369.0,Amazon,1210,0.5,150,1210,0,2,Competitive,0,4,5141,9,5141_Amazon,bb2020,1.0,1.0,1.0,2021,2021-48,2021-11-29
3,4347797,2021-11-30,1054766.0,79054.0,Human,1270,1.0,150,1270,0,2,Competitive,0,4,4964,6,4964_Human,bb2020,1.0,2.0,2.0,2021,2021-48,2021-11-29
4,4347796,2021-11-30,1052653.0,18576.0,Elven Union,1370,1.0,70,1370,0,2,Competitive,0,4,4961,6,4961_Elven Union,bb2020,2.0,2.0,2.0,2021,2021-48,2021-11-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263013,4216262,2020-08-01,995327.0,238300.0,Skink,980,1.0,20,1000,0,5,Secret League,10263,303,395,1,395_Skink,bb2016,NaN,NaN,NaN,2020,2020-31,2020-08-03
263014,4216261,2020-08-01,994045.0,134383.0,Necromantic Horror,1230,1.0,0,1230,0,10,Blackbox,0,1,53,13,53_Necromantic Horror,bb2016,2.0,2.0,2.0,2020,2020-31,2020-08-03
263015,4216260,2020-08-01,988905.0,250129.0,Nurgle,1680,1.0,290,1680,0,10,Blackbox,0,1,55,105,55_Nurgle,bb2016,2.0,2.0,2.0,2020,2020-31,2020-08-03
263016,4216259,2020-08-01,967266.0,250588.0,Human,920,1.0,20,940,0,10,Blackbox,0,1,50,3,50_Human,bb2016,1.0,2.0,2.0,2020,2020-31,2020-08-03
